<a href="https://colab.research.google.com/github/lmoss/onesharp/blob/main/coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding words by numbers

## Shortlex ordering on words
### Here are definitions of the length function on numbers. We also have the function ```s(n)``` giving the nth word in the shortlex order on words, starting with n = 0.  We have the inverse of ```s``` also.

We can check that ```s``` and ```s_inverse``` really are inverses.

In [ ]:
s_inverse(s(4234))

In [2]:
for n in range(256):
  print((n,s(n), s_inverse(s(n))))
 

NameError: ignored